In [2]:
from __future__ import print_function
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as plt
import subprocess

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
def print_top_words(tf_idf, feature_names, n_top_words):
    for topic_idx, topic in enumerate(tf_idf):
        #print(topic.shape)
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [6]:
'''
w = len('Abstract')
pids = filtered.loc[filtered.ABSTRACT==' '].index
for p in pids:
    t = filtered.loc[p,'TITLE']
    print (t)
    cmd = "python /home/yu/gits/scholar/scholar.py -c 1 -t --phrase '"+t+"'"+" --csv"
    print (cmd)
    result = subprocess.check_output(cmd, shell=True)
    print (result)
    filtered.loc[p,'ABSTRACT'] = result.split('|')[-1][w:]
    break
'''

'\nw = len(\'Abstract\')\npids = filtered.loc[filtered.ABSTRACT==\' \'].index\nfor p in pids:\n    t = filtered.loc[p,\'TITLE\']\n    print (t)\n    cmd = "python /home/yu/gits/scholar/scholar.py -c 1 -t --phrase \'"+t+"\'"+" --csv"\n    print (cmd)\n    result = subprocess.check_output(cmd, shell=True)\n    print (result)\n    filtered.loc[p,\'ABSTRACT\'] = result.split(\'|\')[-1][w:]\n    break\n'

In [81]:
publications = pd.read_csv('filtered_outputs.csv')
people = pd.read_csv('people_orgs.csv')

In [87]:
publications['topic'] = -1

In [64]:
topic_years = pd.DataFrame(columns=['y'+str(y) for y in range(2008,2014)])

,y2008,y2009,y2010,y2011,y2012


In [96]:
tf_vectorizer = CountVectorizer(max_df=0.7, min_df=2,
                                max_features=10000,
                                stop_words='english')

In [97]:
T = 50
lda = LatentDirichletAllocation(n_topics=T, max_iter=30,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,
                                n_jobs=1)

In [99]:
for year in range(2008,2014):
    print(year)
    filtered = publications.loc[publications.PUBLICATION_YEAR==year]
    docs = map(lambda i: filtered.TITLE[i]+filtered.ABSTRACT[i], filtered.index)
    
    doc_word_mtrx = tf_vectorizer.fit_transform(docs)
    for w in tf_vectorizer.vocabulary_.keys():
        if len(w) < 3 or any(x.isdigit() for x in w):
            del tf_vectorizer.vocabulary_[w]
    doc_word_mtrx = tf_vectorizer.transform(docs)
    
    df = reduce(lambda x,y:x+y, doc_word_mtrx)
    df = df.toarray()
    df[df==0]=np.max(df)
    
    doc_topic=lda.fit_transform(doc_word_mtrx)
    dp = np.argsort(doc_topic,1)[:,-1]
    tp_doc_counts=np.zeros(T)
    for t in range(T):
        tp_doc_counts[t] = (dp==t).sum()
        
    trank = tp_doc_counts.argsort()
    n_top_words = 50
    tf_feature_names = tf_vectorizer.get_feature_names()
    for t in range(T):
        topic_years.loc[t,'y'+str(year)] = ",".join(tf_feature_names[k] for k in lda.components_[trank[-t-1]].argsort()[:-n_top_words - 1:-1])
        publications.loc[filtered.index[dp==trank[-t-1]],'topic'] = t

2008
2009
2010
2011
2012
2013


In [108]:
topic_years.to_csv('topic_years.csv')

In [111]:
people.PERSON_ID.unique().shape

(3263,)

In [115]:
people_topic = pd.merge(publications, people, on='PUBLICATION_ID')

In [129]:
topic_orgs = pd.DataFrame()
for year in range(2008,2014):
    #topic_orgs[year] = {}
    ycol = 'y'+str(year)
    for t in range(T):
        topic_orgs.loc[t,ycol+'_num_people'] = people_topic.loc[(people_topic.PUBLICATION_YEAR==year)&(people_topic.topic==t)].PERSON_ID.unique().shape[0]
        topic_orgs.loc[t,ycol+'_num_orgs'] = people_topic.loc[(people_topic.PUBLICATION_YEAR==year)&(people_topic.topic==t)].ORGANISATION_CODE.unique().shape[0]
        topic_orgs.loc[t,ycol+'_num_docs'] = people_topic.loc[(people_topic.PUBLICATION_YEAR==year)&(people_topic.topic==t)].PUBLICATION_ID.unique().shape[0]
        #topic_orgs[year][t]={'num_people':num_people,'num_orgs':num_orgs,'num_docs':num_docs}
        

In [132]:
topic_orgs.to_csv('topic_orgs.csv',index=False,float_format='%d')

In [122]:
people_topic.head()

,PUBLICATION_ID,TITLE,TYPE_NO,TYPE_x,PUBLICATION_DAY,PUBLICATION_MONTH,PUBLICATION_YEAR,KEYWORDS,ABSTRACT,authors,...,SURNAME,ORGANISATION_CODE,TYPE_y,JOB_TITLE,START_DATE,END_DATE,PARENT_CODE,ORGANISATION_TYPE,FULL_NAME,GRANDPARENT_CODE
0,89664436,Crossing the Black Atlantic for Africa or How ...,5005,contributiontobookanthology/chapter,0.0,0.0,2012,0,0,67324,...,Peters,MODL,staff,Lecturer in Latin American History (Political ...,01/09/2016,31/12/2199,FOAT,ACADEMIC SCHOOL,School of Modern Languages,ACAD
1,55265799,Central venous catheters and outcomes in hemod...,5005,contributiontobookanthology/chapter,0.0,11.0,2012,0,<p>A significant proportion of incident and pr...,58056,...,Power,SOCS,staff,Senior Lecturer,01/01/2015,31/12/2017,FMDY,ACADEMIC SCHOOL,School of Clinical Sciences,ACAD
2,53490655,Communication as information use,5005,contributiontobookanthology/chapter,1.0,1.0,2011,0,<p>Introduction Uncertainty is an unavoidable ...,"1968, 12503",...,McNamara,MATH,staff,Emeritus Professor,01/01/2016,31/12/2199,FSCI,ACADEMIC SCHOOL,School of Mathematics,ACAD
3,53490655,Communication as information use,5005,contributiontobookanthology/chapter,1.0,1.0,2011,0,<p>Introduction Uncertainty is an unavoidable ...,"1968, 12503",...,McNamara,MATH,staff,Emeritus Professor,01/01/2016,31/12/2199,FSCI,ACADEMIC SCHOOL,School of Mathematics,ACAD
4,53490655,Communication as information use,5005,contributiontobookanthology/chapter,1.0,1.0,2011,0,<p>Introduction Uncertainty is an unavoidable ...,"1968, 12503",...,McNamara,MATH,staff,Emeritus Professor,01/01/2016,31/12/2199,FSCI,ACADEMIC SCHOOL,School of Mathematics,ACAD


In [120]:
filtered.shape

(5219, 13)